# Initial implementation of PIICM

Will make a stab at implementing the original version of PIICM in the newest version of gpytorch. There are a few ingredients that are needed

1. The kernel  -- we need to construct a permutation invariant covariance function.
    1. Needs to return objects with custom .matmul routines, custom log_det calculations etc. This was done in the original implementation by extending the `LazyTensor` class, which is no longer available. Will need to work with the `LinearOperators` instead, but hopefully should be rather straight-forward.
2. The likelihood -- we need a multi-task normal likelihood that allows us to add a observation specific noise term
3. Model -- putting these two together and maximizing the marginal likelihood using CG for the inverse solve and a custom log-determinant calculation involving Weyl's inequality.

It needs to all play nice with the current classes, extending were needed but otherwise not interferring with gpytorch.

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import synpred

ModuleNotFoundError: No module named 'synpred'